In [ ]:
!pip install yolov5

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [1]:
#import yolov5
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from PIL import Image, ImageDraw
import time
from torchvision.utils import draw_bounding_boxes
#from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, AutoImageProcessor, DefaultDataCollator
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision
import tqdm
import os

In [ ]:
!pip freeze > requirements.txt.neuro

In [2]:
from pathlib import Path

In [3]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [5]:
%cd /content/drive/My Drive//Nuclear IT Hack

/content/drive/My Drive/Nuclear IT Hack


In [ ]:
!unzip dataset.zip

In [ ]:
!unzip dataset_hack

#Считывание исходного датасета

In [ ]:
smokers_path = sorted(list(map(str, Path("./smokers").glob("*.jpg"))))
non_smokers_path = sorted(list(map(str, Path("./non_smokers").glob("*.jpg"))))
labels = []
for i, path in enumerate(smokers_path):
    with Image.open(path) as img:
        if i == 0:
            img = img.resize((640, 640))
            data = np.asarray(img).reshape(1, 640,640,3)
            labels.append("smoker")
        else:
            img = img.resize((640, 640))
            data = np.concatenate((data, np.asarray(img).reshape(1,640,640,3)), axis=0)
for path in non_smokers_path:
    with Image.open(path) as img:
        img = img.resize((640, 640))
        data = np.concatenate((data, np.asarray(img).reshape(1, 640,640,3)), axis=0)
        labels.append("non_smoker")

In [ ]:
DATA_MODES = ["train", "val", "test"]
RESCALE_SIZE = 299
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
file_path_smoke = list(map(str, Path("./smokers").glob("*.jpg")))
file_path_non = list(map(str, Path("./people").glob("*.jpg")))
train_val_labels = [1] * len(file_path_smoke) + [0] * len(file_path_non)
file_path = file_path_smoke + file_path_non

In [ ]:
test_files_smoke = list(map(str, Path("./dataset/smokers").glob("*.jpg")))
test_files_non = list(map(str, Path("./dataset/people").glob("*.jpg")))
test_files = test_files_smoke + test_files_non
test_labels = [1] * len(test_files_smoke) + [0] * len(test_files_non)

In [ ]:
file_path += test_files
train_val_labels += test_labels

In [ ]:
class SmokersDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode, labels):
        super().__init__()
        # список файлов для загрузки
        self.files = files
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
        self.labels = labels

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        if self.mode != "test":
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                transforms.RandomHorizontalFlip(p=0.5),
            ])
            x = self.load_sample(self.files[index])
            x = self._prepare_sample(x)
            x = np.array(x / 255, dtype='float32')
            if list(x.shape) != [RESCALE_SIZE, RESCALE_SIZE, 3]:
                print(self.files[index], x.shape)
            x = transform(x)
        else:
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])
            x = self.load_sample(self.files[index])
            x = self._prepare_sample(x)
            x = np.array(x / 255, dtype='float32')

        label = self.labels[index]
        return x, label

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [ ]:
from sklearn.model_selection import train_test_split

train_files, val_files, train_labels, val_labels = train_test_split(file_path, train_val_labels, test_size=0.25, \
                                          stratify=train_val_labels)

train_files += test_files
train_labels += test_labels

#Подготовка модели детекции

In [ ]:
model = yolov5.load('yolov5l6.pt')

In [ ]:
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.max_det = 1000  # maximum number of detections per image
model.classes = 0

In [ ]:
tic = time.time()
out = model("/content/example1.jpg")
toc = time.time()
print(f"Prosses time: {(toc - tic) * 1000} ms")

Prosses time: 267.2383785247803 ms


In [ ]:
crop_img = out.crop(save=False)

In [ ]:
out.xyxy[0][0][:4].reshape(1,4).shape

torch.Size([1, 4])

In [ ]:
bbox = draw_bounding_boxes(torch.tensor(np.moveaxis(data[0], 2, 0)), out.xyxy[0][0][:4].reshape(1,4))

In [ ]:
bbox.shape

torch.Size([3, 640, 640])

In [ ]:
image = Image.fromarray(np.moveaxis(np.array(bbox), 0, 2))

In [ ]:
for i in range(data.shape[0]):
    out = model(data[i])


#Тренировка классификатора

In [ ]:
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
labels = ['smoker', 'non_smoker']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
class_model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    from_pt=False
)

In [ ]:
class_model.init_weights()

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#InceptionV3

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs).logits
        #outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)

    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [ ]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    model.eval()
    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).detach().cpu().numpy()
        processed_size += inputs.size(0)
        # print(f"labels: {labels}")
        # print(f"preds: {preds}")
    val_loss = running_loss / processed_size
    val_acc = running_corrects / processed_size
    return val_loss, val_acc

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    with tqdm.tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))

            pbar_outer.update(1)
            tqdm.tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            torch.cuda.empty_cache()
    return history

In [ ]:
def predict(model, test_loader):
    labels = []
    with torch.no_grad():
        logits = []

        for inputs, label in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            labels += label
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs, labels

In [ ]:
my_inception_v3 = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
my_inception_v3.fc = nn.Linear(2048, 2)

In [ ]:
load_weights("Inception_weights", my_inception_v3)

In [ ]:
for params in list(my_inception_v3.parameters())[:-25]:
    params.requires_grad = False

In [ ]:
my_inception_v3.to(DEVICE)
1

1

In [ ]:
val_dataset = SmokersDataset(val_files, mode='val', labels=val_labels)

train_dataset = SmokersDataset(train_files, mode='train', labels=train_labels)

In [ ]:
history = train(train_dataset, val_dataset, model=my_inception_v3, epochs=5, batch_size=32)

epoch:   0%|          | 0/5 [00:00<?, ?it/s]

loss 0.438278631362694


epoch:  20%|██        | 1/5 [00:14<00:59, 14.95s/it]


Epoch 001 train_loss: 0.4383     val_loss 0.5888 train_acc 0.7881 val_acc 0.8421
loss 0.1173021195345367


epoch:  40%|████      | 2/5 [00:30<00:46, 15.34s/it]

...


Epoch 003 train_loss: 0.0623     val_loss 0.7510 train_acc 0.9834 val_acc 0.8289
loss 0.021585328837498925


epoch:  80%|████████  | 4/5 [01:01<00:15, 15.28s/it]


Epoch 004 train_loss: 0.0216     val_loss 0.8145 train_acc 0.9967 val_acc 0.7763
loss 0.00969709816160581


epoch: 100%|██████████| 5/5 [01:16<00:00, 15.22s/it]


Epoch 005 train_loss: 0.0097     val_loss 0.8150 train_acc 1.0000 val_acc 0.7632


In [ ]:
loss, acc, val_loss, val_acc = zip(*history)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 9))
ax[0].plot(loss, label="train_loss")
ax[0].plot(val_loss, label="val_loss")
ax[0].legend(loc='best')
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("loss")

ax[1].plot(acc, label="train_acc")
ax[1].plot(val_acc, label="val_acc")
ax[1].legend(loc='best')
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("Accuracy")
plt.show()

In [ ]:
test_dataset = SmokersDataset(test_files, mode="test", labels=test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
val_dataloader = DataLoader(val_dataset, batch_size=16)
probs_ims, true_labels = predict(my_inception_v3, val_dataloader)
y_pred = np.argmax(probs_ims,-1)

In [ ]:
correct = np.sum(np.array(true_labels) == np.array(y_pred)) / len(y_pred)
correct

0.8947368421052632

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(true_labels, y_pred, average='micro')

print("F1-оценка:", f1)

F1-оценка: 0.8947368421052632


#AlexNet

In [ ]:
my_alex = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
my_alex

In [ ]:
my_alex.classifier[4] = nn.Linear(4096, 2048)
my_alex.classifier[6] = nn.Linear(2048, 2)

In [ ]:
len(list(my_alex.parameters()))

16

In [ ]:
for params in list(my_inception_v3.parameters())[:-3]:
    params.requires_grad = False

In [ ]:
my_alex.to(DEVICE)
1

1

In [ ]:
history = train(train_dataset, val_dataset, model=my_alex, epochs=10, batch_size=32)

In [ ]:
loss, acc, val_loss, val_acc = zip(*history)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 9))
ax[0].plot(loss, label="train_loss")
ax[0].plot(val_loss, label="val_loss")
ax[0].legend(loc='best')
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("loss")

ax[1].plot(acc, label="train_acc")
ax[1].plot(val_acc, label="val_acc")
ax[1].legend(loc='best')
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("Accuracy")
plt.show()

In [ ]:
len(val_files)

46

In [ ]:
idxs = list(range(val_dataset.len_))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]
actual_labels = [val_dataset[id][1] for id in idxs]
probs_ims = predict(my_alex, imgs)
y_pred = np.argmax(probs_ims,-1)

In [ ]:
correct = np.sum(np.array(actual_labels) == np.array(y_pred)) / len(y_pred)
correct

0.7608695652173914

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(actual_labels, y_pred, average='micro')

print("F1-оценка:", f1)

F1-оценка: 0.7608695652173914


#Инференс

In [ ]:
classificator = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
#Использовать model.parameters()

In [ ]:
def save_weights(name, model):
    if not os.path.exists(name):
        os.mkdir(name)
    for i, params in enumerate(model.parameters()):
        torch.save(params, f"{name}/layer_{i}.pt")

def load_weights(dir_name, model):
    paths = sorted(list(map(str, Path(f"./{dir_name}").glob("*.pt"))))
    for i, params in enumerate(model.parameters()):
        params = torch.load(paths[i])

In [ ]:
save_weights("Inception_weights", my_inception_v3)

In [ ]:
load_weights("Inception_weights", classificator)

In [ ]:
detector = yolov5.load('yolov5s.pt')

detector.conf = 0.25  # NMS confidence threshold
detector.iou = 0.45  # NMS IoU threshold
detector.agnostic = False  # NMS class-agnostic
detector.multi_label = False  # NMS multiple labels per box
detector.max_det = 30  # maximum number of detections per image
detector.classes = 0

classificator  = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
classificator.fc = nn.Linear(2048, 2)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
for params in list(classificator.parameters())[:-7]:
    params.requires_grad = False

In [ ]:
example_image = Image.open(val_files[1])
example_image

In [ ]:
example_image = Image.open(train_files[1])

result = detector(example_image)

In [ ]:
def load_weights(dir_name, model):
    paths = sorted(list(map(str, Path(f"./{dir_name}").glob("*.pt"))))
    for i, params in enumerate(model.parameters()):
        params = torch.load(paths[i])

In [ ]:
detector = yolov5.load('yolov5s.pt')

detector.conf = 0.25  # NMS confidence threshold
detector.iou = 0.45  # NMS IoU threshold
detector.agnostic = False  # NMS class-agnostic
detector.multi_label = False  # NMS multiple labels per box
detector.max_det = 30  # maximum number of detections per image
detector.classes = 0

classificator  = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
classificator.fc = nn.Linear(2048, 2)

In [ ]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
        max_prob = np.max(probs)
        y_pred = np.argmax(probs,-1)
    return y_pred, max_prob

In [ ]:
def analize(example_image):
    prediction = []
    probability = []
    bboxes = []
    result = detector(example_image)
    result = result.crop(save=False)
    n_all_people = len(result)
    for imgs in result:
        if float(imgs['conf'].cpu().numpy()) >= 0.85:
            bbox = imgs['box']
            bbox = [float(val.cpu().numpy()) for val in bbox]
            example_image1 = example_image.crop(bbox)
            example_image1 = example_image.resize((RESCALE_SIZE, RESCALE_SIZE))
            example_image1 = np.array(np.array(example_image1) / 255, dtype='float32')
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])
            example_image1 = transform(example_image)

            pred, prop = predict_one_sample(classificator.to(DEVICE), example_image1.unsqueeze(0), device=DEVICE)
            if pred[0] == 1:
                bboxes.append(bbox)
                prediction.append(pred[0])
                probability.append(prop)

    #torchvision.utils.draw_bounding_boxes(torch.tensor(np.array(example_image)), boxes=bboxes, labels=[f"smoker_{i}; p: {prob}" for i, prob in enumerate(probability)])
    draw = ImageDraw.Draw(example_image)
    for bbox in bboxes:
        draw.rectangle(tuple(bbox), outline="red", width=3)

    return example_image, n_all_people, len(prediction)

In [ ]:
image = Image.open("/content/ex1.jpg")
image, all_people, n_smokers, pred, prop = analize(image)
image

ValueError: Only grayscale and RGB images are supported

In [ ]:
correct = np.sum(np.array(test_labels) == np.array(prediction)) / len(prediction)
correct

38.0

In [ ]:
test_labels[20]

1

In [ ]:
prediction

[1]

In [ ]:
probability

[0.83096653]